In [1]:
import pandas as pd
import numpy as np
from scipy.linalg import sqrtm

from scripts.ukf_utils import wmean, wcov, unscentedTrans

import GPy
from scripts.process import ContactPointTracker
import plotly.graph_objects as go

%load_ext autoreload
%autoreload 2

In [2]:
from stl import mesh
from scipy.spatial import cKDTree

# Tracking on object surfaces consists of multiple sweep where each sweep is recorded in a separate file. This function loads 
# the tracking from all the sweeps into one file to display them together.
def aggregate_object_scan(fm_alpha, var, R, get_track_data_fn, trial_range, mesh_file):
    kern = GPy.kern.ThinPlate(3, variance=var, R=R)

    obj_mesh = mesh.Mesh.from_file(mesh_file)
    vertices, I, J, K = stl2mesh3d(obj_mesh)
    vertices[:,2] = -vertices[:,2]
    vertices = (vertices + np.array([0.5,12.5,-0.5]))

    plot_array= []
    distance_array = []
    points_array = np.zeros((0,3))
    for i in trial_range:
        tracker = ContactPointTracker(fm_alpha=fm_alpha, fm_stride=5, 
                                      sigma_process=2e-5, sigma_process_init=1, 
                                      mu0=np.array([-10,-30,0]))
        tracker.init_calibration_model(calibration_fn='../data/all_datasets/calibration/tip_calib_May_04.csv',
                                    input_ref=np.array([[64.30451325,98.0687005,16.1023935]]),
                                    kernel=kern)

        tracker.initialize(track_data_fn=get_track_data_fn(i),
                                    offset_mode='from_data')

        x_ukf, sig_ukf, additional_data = tracker.run_tracker(track_mode='static')

        x_est = x_ukf.T
        x_est[:,2] = -x_est[:,2]
        x_err = additional_data['X_t']-x_est
        x_err[:,2] = -x_err[:,2]

        selection_arr_np = np.where(np.array(additional_data['selection_arr']))[0]
        x_err_sel = x_err[selection_arr_np,:]

        sig_ukf_sel = sig_ukf[:,:,selection_arr_np]

        ev_array = np.zeros(sig_ukf_sel.shape[2])

        for j in range(sig_ukf_sel.shape[2]):
            ev_array[j] = np.linalg.eig(sig_ukf_sel[:,:,j])[0][1]
        x_err_filt = x_err_sel[np.where(ev_array < 1)[0],:]
        points_array = np.concatenate((points_array, x_err_filt[15:-1,:]), axis=0)

    # Calculate distance between points and the object mesh
    tree = cKDTree(vertices)
    dist, _ = tree.query(points_array)
    dist = np.abs(dist)
    distance_array.append(dist)

    # Calculate the mean and covariance of the distance
    dist_mean = np.mean(dist)
    dist_cov = np.cov(dist)
    dist_min = np.min(dist)
    dist_max = np.max(dist)
    dist_std = np.std(dist)
    print(dist_mean, dist_std, dist_min, dist_max, dist_cov)

    plot_array.append(go.Scatter3d(x=points_array[:,0], y=points_array[:,1],z=points_array[:,2],
                                    name='lines', 
                                    marker=dict(size=5, color=dist,colorscale='Viridis', 
                                                cmin=0,cmax=1.8,
                                                colorbar=dict(thickness=10), 
                                                opacity=0.8),
                                    line=dict(color='rgba(0,0,0,0)'),))
    return plot_array, points_array, distance_array, vertices, I, J, K

# Load mesh

In [3]:
def stl2mesh3d(stl_mesh):
    # stl_mesh is read by nympy-stl from a stl file; it is  an array of faces/triangles (i.e. three 3d points) 
    # this function extracts the unique vertices and the lists I, J, K to define a Plotly mesh3d
    p, q, r = stl_mesh.vectors.shape #(p, 3, 3)
    # the array stl_mesh.vectors.reshape(p*q, r) can contain multiple copies of the same vertex;
    # extract unique vertices from all mesh triangles
    vertices, ixr = np.unique(stl_mesh.vectors.reshape(p*q, r), return_inverse=True, axis=0)
    I = np.take(ixr, [3*k for k in range(p)])
    J = np.take(ixr, [3*k+1 for k in range(p)])
    K = np.take(ixr, [3*k+2 for k in range(p)])
    return vertices, I, J, K

# Double Square Fixture

In [4]:
# Tracking on object surfaces consists of multiple sweep where each sweep is recorded in a separate file. This loads the tracking 
# from all the sweeps into one file to display them together.
plot_array, points_array, distance_array, vertices, I, J, K = aggregate_object_scan(1.2, 5.0, 100, 
                      lambda i: f'../data/all_datasets/test/tip_test_05_19_23/reference_double_squaretip_test_scan_object_May_19_trial{i}.csv',
                     range(0,5), 
                     mesh_file='../data/3d_data/SquareFixtureDense.stl')

0.4908130510370351 0.20403048245933908 0.03798448516695886 1.1965358585973693 0.04168868153203002


In [5]:
# Load the STLs of the reference object shapes to be displayed in 3D
from stl import mesh
obj_mesh = mesh.Mesh.from_file('../data/3d_data/SquareFixture.stl')
vertices, I, J, K = stl2mesh3d(obj_mesh)
vertices[:,2] = -vertices[:,2]
x, y, z = (vertices + np.array([0.5,12.5,-0.5])).T

In [6]:
# x, y, z = (vertices + np.array([0.5,12.5,0.5])).T
colorscale= [[0, '#e5dee5'], [1, '#e5dee5']]   
# x, y, z = (vertices).T

mesh3D = go.Mesh3d(
            x=x,
            y=y,
            z=z,
            i=I,
            j=J,
            k=K,
            flatshading=True,
            colorscale=colorscale, 
            intensity=z, 
            opacity=0.8,
            name='Double Square',
            showscale=False)

title = "Mesh of reference object"
layout = go.Layout(title_text=title, title_x=0.5,
                   font_color='black',
            width=800,
            height=800,
            showlegend=False,
       #      coloraxis = dict(colorscale='Viridis', 
                            # colorbar=dict(thickness=10),),
            scene_camera=dict(eye=dict(x=1.5, y=-1.25, z=1.2)),)
       #      scene_xaxis_visible=False,
       #      scene_yaxis_visible=False,
       #      scene_zaxis_visible=False)

fig = go.Figure(data=[mesh3D]+plot_array, layout=layout)
fig.data[0].update(lighting=dict(ambient= 0.18,
                                 diffuse= 1,
                                 fresnel=  .1,
                                 specular= 1,
                                 roughness= .1,
                                 facenormalsepsilon=0))
fig.data[0].update(lightposition=dict(x=3000,
                                      y=3000,
                                      z=10000))
fig.update_layout(scene = dict(
               xaxis = dict(
                    backgroundcolor="rgba(0, 0, 0,0)",
                    gridcolor="gray",
                    showbackground=True,
                    zerolinecolor="gray",),
               yaxis = dict(
               backgroundcolor="rgba(0, 0, 0,0)",
               gridcolor="gray",
               showbackground=True,
               zerolinecolor="gray"),
               zaxis = dict(
               backgroundcolor="rgba(0, 0, 0,0)",
               gridcolor="gray",
               showbackground=True,
               zerolinecolor="gray",),),
)
fig.update_layout(font=dict(size=14))
fig.show()

# Cup Ref Object

In [9]:
# Tracking on object surfaces consists of multiple sweep where each sweep is recorded in a separate file. This loads the tracking 
# from all the sweeps into one file to display them together.
plot_array, points_array, distance_array, vertices, I, J, K = aggregate_object_scan(1.2, 5.0, 100, 
                      lambda i: f'../data/all_datasets/test/tip_test_05_19_23/reference_cuptip_test_scan_object_May_19_trial{i}.csv',
                     range(0,5), 
                     mesh_file='../data/3d_data/CupFixtureDense.stl')

FileNotFoundError: [Errno 2] No such file or directory: '../data/3d_data/CupFixtureDense.stl'

In [ ]:
# Load the STLs of the reference object shapes to be displayed in 3D
from stl import mesh
obj_mesh = mesh.Mesh.from_file('../data/3d_data/CupFixture.stl')
vertices, I, J, K = stl2mesh3d(obj_mesh)
vertices[:,2] = -vertices[:,2]
x, y, z = (vertices + np.array([0.5,12.5,-0.5])).T

In [ ]:
# x, y, z = (vertices).T
colorscale= [[0, '#e5dee5'], [1, '#e5dee5']]   

mesh3D = go.Mesh3d(
            x=x,
            y=y,
            z=z,
            i=I,
            j=J,
            k=K,
            flatshading=True,
            colorscale=colorscale, 
            intensity=z, 
            opacity=0.8,
            name='Star',
            showscale=False)

title = "Mesh of reference object"
layout = go.Layout(title_text=title, title_x=0.5,
                     font_color='black',
                     width=800,
                     height=800,
                     scene_camera=dict(
                            eye=dict(x=1.25, y=-1.25, z=1),
                            up=dict(x=0, y=0, z=60)
                            ),)
       #      scene_xaxis_visible=False,
       #      scene_yaxis_visible=False,
       #      scene_zaxis_visible=False)

fig = go.Figure(data=[mesh3D]+plot_array, layout=layout)
fig.data[0].update(lighting=dict(ambient= 0.18,
                                 diffuse= 1,
                                 fresnel= .1,
                                 specular= 1,
                                 roughness= .1,
                                 facenormalsepsilon=0))
fig.update_layout(scene = dict(
               xaxis = dict(
                    backgroundcolor="rgba(0, 0, 0,0)",
                    gridcolor="gray",
                    showbackground=True,
                    zerolinecolor="gray",),
               yaxis = dict(
               backgroundcolor="rgba(0, 0, 0,0)",
               gridcolor="gray",
               showbackground=True,
               zerolinecolor="gray"),
               zaxis = dict(
               backgroundcolor="rgba(0, 0, 0,0)",
               gridcolor="gray",
               showbackground=True,
               zerolinecolor="gray",),))
fig.data[0].update(lightposition=dict(x=3000,
                                      y=3000,
                                      z=10000))
fig.update_layout(font=dict(size=14))
fig.show()

# Flat Fixture

In [8]:
# Tracking on object surfaces consists of multiple sweep where each sweep is recorded in a separate file. This loads the tracking 
# from all the sweeps into one file to display them together.
plot_array, points_array, distance_array, vertices, I, J, K = aggregate_object_scan(1.1, 5.0, 100, 
                      lambda i: f'../data/all_datasets/test/tip_test_05_19_23/reference_surfacetip_test_scan_object_May_19_trial{i}.csv',
                     range(0,5), mesh_file='../data/3d_data/FlatFixtureDense.stl')

0.6177845502244155 0.43297531050700205 0.01643925588141334 1.6605378943356899 0.18770982573539016


In [ ]:
# Load the STLs of the reference object shapes to be displayed in 3D
from stl import mesh
obj_mesh = mesh.Mesh.from_file('../data/3d_data/FlatFixture.stl')
vertices, I, J, K = stl2mesh3d(obj_mesh)
vertices[:,2] = -vertices[:,2]
x, y, z = (vertices + np.array([0.5,12.5,-0.5])).T

In [ ]:
x, y, z = (vertices + np.array([0.5,12.5,-0.5])).T
colorscale= [[0, '#e5dee5'], [1, '#e5dee5']]   

mesh3D = go.Mesh3d(
            x=x,
            y=y,
            z=z,
            i=I,
            j=J,
            k=K,
            flatshading=True,
            colorscale=colorscale, 
            intensity=z, 
            opacity=0.8,
            name='Flat',
            showscale=False)


title = "Mesh of reference object"
layout = go.Layout(title_text=title, title_x=0.5,
              font_color='black',
              width=800,
              height=800,
              scene = dict(
                     xaxis = dict(nticks=4, range=[-30,40],),
                     yaxis = dict(nticks=4, range=[-20,50],),
                     zaxis = dict(nticks=4, range=[0,70],),),
              scene_camera=dict(eye=dict(x=1.25, y=-1.25, z=1)),)
       #      scene_xaxis_visible=False,
       #      scene_yaxis_visible=False,
       #      scene_zaxis_visible=False)

fig = go.Figure(data=[mesh3D]+plot_array, layout=layout)
fig.data[0].update(lighting=dict(ambient= 0.18,
                                 diffuse= 1,
                                 fresnel=  .1,
                                 specular= 1,
                                 roughness= .1,
                                 facenormalsepsilon=0))
fig.data[0].update(lightposition=dict(x=3000,
                                      y=3000,
                                      z=10000))
fig.update_layout(scene = dict(
               xaxis = dict(
                    backgroundcolor="rgba(0, 0, 0,0)",
                    gridcolor="gray",
                    showbackground=True,
                    zerolinecolor="gray",),
               yaxis = dict(
               backgroundcolor="rgba(0, 0, 0,0)",
               gridcolor="gray",
               showbackground=True,
               zerolinecolor="gray"),
               zaxis = dict(
               backgroundcolor="rgba(0, 0, 0,0)",
               gridcolor="gray",
               showbackground=True,
               zerolinecolor="gray",),))
fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
    range=[-10,20]
  )
fig.update_layout(font=dict(size=14))
fig.show()

# Cone Fixture 

In [ ]:
# Tracking on object surfaces consists of multiple sweep where each sweep is recorded in a separate file. This loads the tracking 
# from all the sweeps into one file to display them together.
plot_array, points_array, distance_array, vertices, I, J, K  = aggregate_object_scan(1.1, 1.0, 100, 
                      lambda i: f'../data/all_datasets/test/tip_test_05_27_23/reference_conetip_test_scan_object_May_27_trial{i}.csv',
                     range(0,5), mesh_file='../data/3d_data/ConeFixtureDense.stl')

In [ ]:
# Load the STLs of the reference object shapes to be displayed in 3D
from stl import mesh
obj_mesh = mesh.Mesh.from_file('../data/3d_data/ConeFixture.stl')
vertices, I, J, K = stl2mesh3d(obj_mesh)

In [ ]:
x, y, z = (vertices + np.array([0.5,12.5,-0.5])).T
colorscale= [[0, '#e5dee5'], [1, '#e5dee5']]   

mesh3D = go.Mesh3d(
            x=x,
            y=y,
            z=-z,
            i=I,
            j=J,
            k=K,
            flatshading=True,
            colorscale=colorscale, 
            intensity=z, 
            opacity=0.8,
            name='Flat',
            showscale=False)


title = "Mesh of reference object"
layout = go.Layout(title_text=title, title_x=0.5,
              font_color='black',
              width=800,
              height=800,
              scene = dict(
                     xaxis = dict(nticks=4, range=[-30,40],),
                     yaxis = dict(nticks=4, range=[-20,50],),
                     zaxis = dict(nticks=4, range=[0,70],),),
              scene_camera=dict(eye=dict(x=1.25, y=-1.25, z=1)),)
       #      scene_xaxis_visible=False,
       #      scene_yaxis_visible=False,
       #      scene_zaxis_visible=False)

fig = go.Figure(data=[mesh3D]+plot_array, layout=layout)
# fig = go.Figure(data=plot_array, layout=layout)
fig.data[0].update(lighting=dict(ambient= 0.18,
                                 diffuse= 1,
                                 fresnel=  .1,
                                 specular= 1,
                                 roughness= .1,
                                 facenormalsepsilon=0))
fig.data[0].update(lightposition=dict(x=3000,
                                      y=3000,
                                      z=10000))
fig.update_layout(scene = dict(
               xaxis = dict(
                    backgroundcolor="rgba(0, 0, 0,0)",
                    gridcolor="gray",
                    showbackground=True,
                    zerolinecolor="gray",),
               yaxis = dict(
               backgroundcolor="rgba(0, 0, 0,0)",
               gridcolor="gray",
               showbackground=True,
               zerolinecolor="gray"),
               zaxis = dict(
               backgroundcolor="rgba(0, 0, 0,0)",
               gridcolor="gray",
               showbackground=True,
               zerolinecolor="gray",),))
fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
    range=[-10,20]
  )
fig.update_layout(font=dict(size=14))
fig.show()

# Star Fixture

In [ ]:
# Tracking on object surfaces consists of multiple sweep where each sweep is recorded in a separate file. This loads the tracking 
# from all the sweeps into one file to display them together.
plot_array = aggregate_object_scan(1.1, 1.0, 100, 
                      lambda i: f'../data/all_datasets/test/tip_test_05_19_23/reference_startip_test_scan_object_May_19_trial{i}.csv',
                     range(0,5))

In [ ]:
# Load the STLs of the reference object shapes to be displayed in 3D
from stl import mesh
obj_mesh = mesh.Mesh.from_file('../data/3d_data/StarFixture2.stl')
vertices, I, J, K = stl2mesh3d(obj_mesh)

In [ ]:
x, y, z = (vertices + np.array([0.5,12.5,-0.5])).T
colorscale= [[0, '#e5dee5'], [1, '#e5dee5']]   

mesh3D = go.Mesh3d(
            x=x,
            y=y,
            z=z,
            i=I,
            j=J,
            k=K,
            flatshading=True,
            colorscale=colorscale, 
            intensity=z, 
            opacity=0.8,
            name='Star',
            showscale=False)


title = "Mesh of reference object"
layout = go.Layout(title_text=title, title_x=0.5,
                   font_color='black',
            width=800,
            height=800,
            scene_camera=dict(eye=dict(x=1.25, y=-1.25, z=1)),)
       #      scene_xaxis_visible=False,
       #      scene_yaxis_visible=False,
       #      scene_zaxis_visible=False)

fig = go.Figure(data=[mesh3D]+plot_array, layout=layout)
fig.data[0].update(lighting=dict(ambient= 0.18,
                                 diffuse= 1,
                                 fresnel=  .1,
                                 specular= 1,
                                 roughness= .1,
                                 facenormalsepsilon=0))
fig.data[0].update(lightposition=dict(x=3000,
                                      y=3000,
                                      z=10000))
fig.show()

# Step Fixture 

In [ ]:
# Tracking on object surfaces consists of multiple sweep where each sweep is recorded in a separate file. This loads the tracking 
# from all the sweeps into one file to display them together.
plot_array = aggregate_object_scan(1.1, 1.0, 100, 
                      lambda i: f'../data/all_datasets/test/tip_test_05_27_23/reference_step_5mm_farthertip_test_scan_object_May_27_trial{i}.csv',
                     range(0,5))

In [ ]:
# Load the STLs of the reference object shapes to be displayed in 3D
from stl import mesh
obj_mesh = mesh.Mesh.from_file('../data/3d_data/StepFixture.stl')
vertices, I, J, K = stl2mesh3d(obj_mesh)

In [ ]:
x, y, z = (vertices + np.array([0.5,12.5,-0.5])).T
colorscale= [[0, '#e5dee5'], [1, '#e5dee5']]   

mesh3D = go.Mesh3d(
            x=x,
            y=y,
            z=-z,
            i=I,
            j=J,
            k=K,
            flatshading=True,
            colorscale=colorscale, 
            intensity=z, 
            opacity=0.8,
            name='Flat',
            showscale=False)


title = "Mesh of reference object"
layout = go.Layout(title_text=title, title_x=0.5,
              font_color='black',
              width=800,
              height=800,
              scene = dict(
                     xaxis = dict(nticks=4, range=[-30,40],),
                     yaxis = dict(nticks=4, range=[-20,50],),
                     zaxis = dict(nticks=4, range=[0,70],),),
              scene_camera=dict(eye=dict(x=1.25, y=-1.25, z=1)),)
       #      scene_xaxis_visible=False,
       #      scene_yaxis_visible=False,
       #      scene_zaxis_visible=False)

fig = go.Figure(data=[mesh3D]+plot_array, layout=layout)
# fig = go.Figure(data=plot_array, layout=layout)
fig.data[0].update(lighting=dict(ambient= 0.18,
                                 diffuse= 1,
                                 fresnel=  .1,
                                 specular= 1,
                                 roughness= .1,
                                 facenormalsepsilon=0))
fig.data[0].update(lightposition=dict(x=3000,
                                      y=3000,
                                      z=10000))
fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
    range=[-10,20]
  )
fig.show()